In [1]:
import os 
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.f' % x)
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.chdir('C:\\Users\\rhian\\Downloads\\CYPLAN 255\\AP-Exams\\Data')
os.getcwd()

'C:\\Users\\rhian\\Downloads\\CYPLAN 255\\AP-Exams\\Data'

In [3]:
ap_cali = pd.read_csv('AP_Cali_cleaned.csv')
ap_cali = ap_cali.drop(['Unnamed: 0'], axis=1)
ap_cali

,ncessch,year,leaid,race,sex,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore
0,40032402160,2011,400324,1,1,NaN,NaN,NaN,NaN
1,40032402160,2011,400324,2,1,NaN,NaN,NaN,NaN
2,40032402160,2011,400324,3,1,NaN,NaN,NaN,NaN
3,40032402160,2011,400324,4,1,NaN,NaN,NaN,NaN
4,40032402160,2011,400324,5,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
960163,60573014153,2017,605730,4,99,NaN,NaN,NaN,NaN
960164,60573014153,2017,605730,5,99,NaN,NaN,NaN,NaN
960165,60573014153,2017,605730,6,99,NaN,NaN,NaN,NaN
960166,60573014153,2017,605730,7,99,NaN,NaN,NaN,NaN


# Investigate race/sex Features

In [4]:
os.chdir('C:\\Users\\rhian\\Downloads\\CYPLAN 255\\AP-Exams\\Data\\Raw Data and Collection')
os.getcwd()

'C:\\Users\\rhian\\Downloads\\CYPLAN 255\\AP-Exams\\Data\\Raw Data and Collection'

In [5]:
apibgt_2011 = pd.read_csv('APIBGT_2011.csv')
apibgt_2013 = pd.read_csv('APIBGT_2013.csv')
apibgt_2015 = pd.read_csv('APIBGT_2015.csv')
apibgt_2017 = pd.read_csv('APIBGT_2017.csv')

apibgt = pd.concat([apibgt_2011, apibgt_2013, 
                    apibgt_2015, apibgt_2017]).reset_index(drop=True) 

# drop rows with letters in id columns 
apibgt = apibgt[apibgt['ncessch'] != "06CC14100001"]

# turn id columns to numeric values 
cols = ['ncessch', 'leaid']
apibgt[cols] = apibgt[cols].apply(pd.to_numeric, axis=1)

apibgt = apibgt.drop(['Unnamed: 0', 'crdc_id', 'fips', 'leaid'], axis=1)

# Drop rows which contain all NaN values in the selected columns
apibgt = apibgt.dropna(how='all', subset=['enrl_IB', 'enrl_AP', 'enrl_AP_science', 
                                 'enrl_AP_math', 'enrl_AP_other', 'enrl_AP_language',
                                 'enrl_gifted_talented'])
apibgt

C:\Users\rhian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\rhian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,year,ncessch,race,sex,enrl_IB,enrl_AP,enrl_AP_science,enrl_AP_math,enrl_AP_other,enrl_AP_language,enrl_gifted_talented
0,2011,40032402160,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,40032402160,2,1,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,40032402160,3,1,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,40032402160,4,1,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,40032402160,5,1,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
960139,2017,60837012539,4,99,NaN,0,0,0,0,NaN,NaN
960140,2017,60837012539,5,99,NaN,1,1,0,0,NaN,NaN
960141,2017,60837012539,6,99,NaN,0,0,0,0,NaN,NaN
960142,2017,60837012539,7,99,NaN,17,5,8,12,NaN,NaN


In [6]:
absent_2013 = pd.read_csv('chronic_absent_2013.csv')
absent_2015 = pd.read_csv('chronic_absent_2015.csv')
# NOTE: these years have different number of available values 
# but their means and sd are p similar 

absent_2011 = absent_2013.copy()
absent_2011['year'] = 2011

absent_2017 = absent_2015.copy()
absent_2017['year'] = 2017

absent = pd.concat([absent_2011, absent_2013, absent_2015, absent_2017]).reset_index(drop=True)

absent = absent.drop(['crdc_id', 'fips', 'leaid'], axis=1)

# Drop rows which contain all NaN values in the selected columns
absent = absent.dropna(how='all', subset=['students_chronically_absent'])

absent

,ncessch,year,race,sex,students_chronically_absent
0,60000103278,2011,5,2,0
1,60000103278,2011,5,1,0
2,60000103278,2011,5,99,0
3,60000103278,2011,4,2,0
4,60000103278,2011,4,1,0
...,...,...,...,...,...
925099,69113710341,2017,7,1,0
925100,69113710341,2017,7,99,0
925101,69113710341,2017,1,2,29
925102,69113710341,2017,1,1,20


In [7]:
discipline_2011 = pd.read_csv('discipline_2011.csv')
discipline_2013  = pd.read_csv('discipline_2013.csv')
discipline_2015 = pd.read_csv('discipline_2015.csv')
discipline_2017 = pd.read_csv('discipline_2017.csv')

discipline = pd.concat([discipline_2011, discipline_2013, 
                        discipline_2015, discipline_2017]).reset_index(drop=True)

discipline = discipline.drop(['Unnamed: 0'], axis=1)

# Drop rows which contain any/all NaN values in the selected columns
discipline = discipline.dropna(how='any', 
                               subset=['ncessch', 'year', 'race', 'sex'])
discipline = discipline.dropna(how='all', 
                               subset=['students_susp_in_sch', 'students_susp_out_sch_single',
                                       'students_susp_out_sch_multiple', 'expulsions_no_ed_serv',
                                       'expulsions_with_ed_serv', 'expulsions_zero_tolerance',
                                       'students_arrested', 'students_referred_law_enforce'])

# Label all negative values in relevant columns as NaN 
discipline[discipline[['transfers_alt_sch_disc']] < 0] = np.nan

# drop rows with letters in id columns 
discipline = discipline[discipline['ncessch'] != "06CC14100001"]

# turn id columns to numeric values 
cols = ['ncessch']
discipline[cols] = discipline[cols].apply(pd.to_numeric, axis=1)

discipline

C:\Users\rhian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ncessch,year,race,sex,students_susp_in_sch,students_susp_out_sch_single,students_susp_out_sch_multiple,expulsions_no_ed_serv,expulsions_with_ed_serv,expulsions_zero_tolerance,students_arrested,students_referred_law_enforce,transfers_alt_sch_disc
0,40032402160,2011,1,1,0,0,0,0,0,0,0,0,NaN
1,40032402160,2011,1,2,0,0,0,0,0,0,0,0,NaN
2,40032402160,2011,1,99,0,0,0,0,0,0,0,0,NaN
3,40032402160,2011,2,1,0,0,0,0,0,0,0,0,NaN
4,40032402160,2011,2,2,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
960187,60573014153,2017,7,2,0,0,0,0,0,0,0,0,0
960188,60573014153,2017,7,99,0,0,0,0,0,0,0,0,0
960189,60573014153,2017,99,1,0,0,0,0,0,0,0,0,0
960190,60573014153,2017,99,2,0,0,0,0,0,0,0,0,0


In [8]:
enrollment = pd.read_csv('enrollment.csv')

enrollment = enrollment.drop(['crdc_id', 'fips', 'leaid'], axis=1)


# Drop rows which contain any/all NaN values in the selected columns
enrollment = enrollment.dropna(how='any', 
                               subset=['ncessch', 'year', 'race', 'sex', 'enrollment_crdc']) 

# drop rows with letters in id columns 
enrollment = enrollment[enrollment['ncessch'] != "06CC14100001"]

# turn id columns to numeric values 
cols = ['ncessch']
enrollment[cols] = enrollment[cols].apply(pd.to_numeric, axis=1)

enrollment

C:\Users\rhian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ncessch,year,race,sex,enrollment_crdc
0,40032402160,2011,1,1,26
1,40032402160,2011,1,2,14
2,40032402160,2011,1,99,40
3,40032402160,2011,2,1,44
4,40032402160,2011,2,2,17
...,...,...,...,...,...
960187,60573014153,2017,7,2,3
960188,60573014153,2017,7,99,5
960189,60573014153,2017,99,1,22
960190,60573014153,2017,99,2,36


In [9]:
harassment_2011 = pd.read_csv('harassment_2011.csv')
harassment_2013  = pd.read_csv('harassment_2013.csv')
harassment_2015 = pd.read_csv('harassment_2015.csv')
harassment_2017 = pd.read_csv('harassment_2017.csv')

harassment = pd.concat([harassment_2011, harassment_2013, 
                        harassment_2015, harassment_2017]).reset_index(drop=True)

harassment = harassment.drop(['crdc_id', 'leaid'], axis=1)

# Drop rows which contain any/all NaN values in the selected columns
harassment = harassment.dropna(how='any', 
                               subset=['ncessch', 'year', 'race', 'sex'])
harassment = harassment.dropna(how='all', 
                               subset=['students_disc_harass_dis', 'students_disc_harass_race',
                                       'students_disc_harass_sex', 'students_report_harass_dis',
                                       'students_report_harass_race', 'students_report_harass_sex'])

# drop rows with letters in id columns 
harassment = harassment[harassment['ncessch'] != "06CC14100001"]

# turn id columns to numeric values 
cols = ['ncessch']
harassment[cols] = harassment[cols].apply(pd.to_numeric, axis=1)

harassment

C:\Users\rhian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\rhian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,year,ncessch,race,sex,students_disc_harass_dis,students_disc_harass_race,students_disc_harass_sex,students_report_harass_dis,students_report_harass_race,students_report_harass_sex
0,2011,40032402160,1,1,0,0,0,0,0,0
1,2011,40032402160,2,1,0,0,0,0,0,0
2,2011,40032402160,3,1,0,0,0,0,0,0
3,2011,40032402160,4,1,0,0,0,0,0,0
4,2011,40032402160,5,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
960187,2017,60573014153,4,99,0,0,0,0,0,0
960188,2017,60573014153,5,99,0,0,0,0,0,0
960189,2017,60573014153,6,99,0,0,0,0,0,0
960190,2017,60573014153,7,99,0,0,0,0,0,0


In [10]:
SAT_2011 = pd.read_csv('SAT_2011.csv')
SAT_2013  = pd.read_csv('SAT_2013.csv')
SAT_2015 = pd.read_csv('SAT_2015.csv')
SAT_2017 = pd.read_csv('SAT_2017.csv')

SAT = pd.concat([SAT_2011, SAT_2013, 
                        SAT_2015, SAT_2017]).reset_index(drop=True)


SAT = SAT.drop(['Unnamed: 0', 'crdc_id', 'leaid'], axis=1)

# Drop rows which contain any/all NaN values in the selected columns
SAT = SAT.dropna(how='any', subset=['ncessch', 'year', 'race', 'sex', 'students_SAT_ACT'])

# drop rows with letters in id columns 
SAT = SAT[SAT['ncessch'] != "06CC14100001"]

# turn id columns to numeric values 
cols = ['ncessch']
SAT[cols] = SAT[cols].apply(pd.to_numeric, axis=1)

SAT

C:\Users\rhian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\rhian\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ncessch,year,race,sex,students_SAT_ACT
0,40032402160,2011,1,1,0
1,40032402160,2011,2,1,0
2,40032402160,2011,3,1,0
3,40032402160,2011,4,1,0
4,40032402160,2011,5,1,0
...,...,...,...,...,...
960187,60573014153,2017,4,99,0
960188,60573014153,2017,5,99,0
960189,60573014153,2017,6,99,0
960190,60573014153,2017,7,99,0


In [11]:
# merge directory with main boy 
ap_features = ap_cali.merge(enrollment, on = ['ncessch', 'year', 'race', 'sex'], how = 'left')
ap_features = ap_features.merge(apibgt, on = ['year', 'ncessch', 'race', 'sex'], how = 'left')
ap_features = ap_features.merge(SAT, on = ['ncessch', 'year', 'race', 'sex'], how = 'left')
ap_features = ap_features.merge(absent, on = ['ncessch', 'year', 'race', 'sex'], how = 'left') 
ap_features = ap_features.merge(harassment, on = ['ncessch', 'year', 'race', 'sex'], how = 'left')
ap_features = ap_features.merge(discipline, on = ['ncessch', 'year', 'race', 'sex'], how = 'left') 

ap_features 

,ncessch,year,leaid,race,sex,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore,enrollment_crdc,enrl_IB,enrl_AP,enrl_AP_science,enrl_AP_math,enrl_AP_other,enrl_AP_language,enrl_gifted_talented,students_SAT_ACT,students_chronically_absent,students_disc_harass_dis,students_disc_harass_race,students_disc_harass_sex,students_report_harass_dis,students_report_harass_race,students_report_harass_sex,students_susp_in_sch,students_susp_out_sch_single,students_susp_out_sch_multiple,expulsions_no_ed_serv,expulsions_with_ed_serv,expulsions_zero_tolerance,students_arrested,students_referred_law_enforce,transfers_alt_sch_disc
0,40032402160,2011,400324,1,1,NaN,NaN,NaN,NaN,26,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
1,40032402160,2011,400324,2,1,NaN,NaN,NaN,NaN,44,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
2,40032402160,2011,400324,3,1,NaN,NaN,NaN,NaN,59,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
3,40032402160,2011,400324,4,1,NaN,NaN,NaN,NaN,20,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
4,40032402160,2011,400324,5,1,NaN,NaN,NaN,NaN,17,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4371167,60573014153,2017,605730,4,99,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4371168,60573014153,2017,605730,5,99,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4371169,60573014153,2017,605730,6,99,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4371170,60573014153,2017,605730,7,99,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
ap_features[(ap_features['ncessch'] == 60140900491) & (ap_features['year'] == 2011) & 
            (ap_features['sex'] == 2) & (ap_features['race'] == 2)] 

,ncessch,year,leaid,race,sex,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore,enrollment_crdc,enrl_IB,enrl_AP,enrl_AP_science,enrl_AP_math,enrl_AP_other,enrl_AP_language,enrl_gifted_talented,students_SAT_ACT,students_chronically_absent,students_disc_harass_dis,students_disc_harass_race,students_disc_harass_sex,students_report_harass_dis,students_report_harass_race,students_report_harass_sex,students_susp_in_sch,students_susp_out_sch_single,students_susp_out_sch_multiple,expulsions_no_ed_serv,expulsions_with_ed_serv,expulsions_zero_tolerance,students_arrested,students_referred_law_enforce,transfers_alt_sch_disc
156745,60140900491,2011,601409,2,2,0,NaN,2,NaN,0,0,0,0,0,0,0,NaN,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


# GETTING RID OF DUPLICATE KEYS 

In [21]:
key_counts = ap_features.groupby(['ncessch', 'year', 'race', 'sex']).size().reset_index().rename(columns={0:'count'})
key_counts[key_counts['count'] > 1]

,ncessch,year,race,sex,count
68904,60171012521,2011,1,1,2
68905,60171012521,2011,1,2,2
68906,60171012521,2011,1,99,2
68907,60171012521,2011,2,1,2
68908,60171012521,2011,2,2,2
...,...,...,...,...,...
921811,69103512016,2017,7,2,64
921812,69103512016,2017,7,99,64
921813,69103512016,2017,99,1,64
921814,69103512016,2017,99,2,64


In [30]:
ap_features = ap_features.groupby(['ncessch', 'year', 'sex', 'race']).max().reset_index()

In [31]:
## Creating indicator column for missing data in students_AP_exam_oneormore
ap_features['AP_oneormore_Present'] = (~ap_features['students_AP_exam_oneormore'].isnull()).astype(int) 
ap_features['AP_none_Present'] = (~ap_features['students_AP_exam_none'].isnull()).astype(int) 

In [32]:
ap_features

,ncessch,year,sex,race,leaid,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore,enrollment_crdc,enrl_IB,enrl_AP,enrl_AP_science,enrl_AP_math,enrl_AP_other,enrl_AP_language,enrl_gifted_talented,students_SAT_ACT,students_chronically_absent,students_disc_harass_dis,students_disc_harass_race,students_disc_harass_sex,students_report_harass_dis,students_report_harass_race,students_report_harass_sex,students_susp_in_sch,students_susp_out_sch_single,students_susp_out_sch_multiple,expulsions_no_ed_serv,expulsions_with_ed_serv,expulsions_zero_tolerance,students_arrested,students_referred_law_enforce,transfers_alt_sch_disc,AP_oneormore_Present,AP_none_Present
0,40032402160,2011,1,1,400324,NaN,NaN,NaN,NaN,26,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0
1,40032402160,2011,1,2,400324,NaN,NaN,NaN,NaN,44,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0
2,40032402160,2011,1,3,400324,NaN,NaN,NaN,NaN,59,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0
3,40032402160,2011,1,4,400324,NaN,NaN,NaN,NaN,20,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0
4,40032402160,2011,1,5,400324,NaN,NaN,NaN,NaN,17,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937939,390059405085,2015,99,4,3900594,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
937940,390059405085,2015,99,5,3900594,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
937941,390059405085,2015,99,6,3900594,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
937942,390059405085,2015,99,7,3900594,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
ap_features.describe()

,ncessch,year,sex,race,leaid,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore,enrollment_crdc,enrl_IB,enrl_AP,enrl_AP_science,enrl_AP_math,enrl_AP_other,enrl_AP_language,enrl_gifted_talented,students_SAT_ACT,students_chronically_absent,students_disc_harass_dis,students_disc_harass_race,students_disc_harass_sex,students_report_harass_dis,students_report_harass_race,students_report_harass_sex,students_susp_in_sch,students_susp_out_sch_single,students_susp_out_sch_multiple,expulsions_no_ed_serv,expulsions_with_ed_serv,expulsions_zero_tolerance,students_arrested,students_referred_law_enforce,transfers_alt_sch_disc,AP_oneormore_Present,AP_none_Present
count,937944,937944,937944,937944,937944,114272,32589,114288,32298,937944,69192,114432,96720,105288,112416,28200,557524,270609,903896,922664,922078,922341,917562,917383,917355,925563,931733,931757,931752,928032,930768,923004,920977,463848,937944,937944
mean,62436762869,2014,34,16,624368,13,37,41,62,105,6,57,18,16,47,8,13,17,13,0,0,0,0,0,0,2,3,2,0,0,0,0,0,0,0,0
std,2452057137,2,46,31,24521,38,50,97,96,236,37,125,46,41,106,21,46,71,36,2,4,4,1,1,1,13,11,7,1,1,1,3,4,1,0,0
min,40032402160,2011,1,1,400324,0,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25%,61187001324,2013,1,3,611870,0,7,0,10,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50%,62271010836,2013,2,4,622710,1,19,2,26,11,0,5,2,2,4,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
75%,63386005308,2017,99,6,633860,8,46,32,73,97,0,49,14,12,40,2,7,2,10,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
max,390059405085,2017,99,99,3900594,1381,809,1565,1274,23010,1375,1965,1418,1522,1771,380,2254,6081,1773,628,1425,1255,228,244,244,1344,469,670,132,226,126,908,917,51,1,1


In [34]:
ap_features.to_csv('AP_Student_Features.csv', index = False)